## Klasyfikacja
Sieć perceptronowa, wielowarstwowa własna

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Wczytanie danych
data = pd.read_csv('star_classification.csv', delimiter=",")

# Wybrane cechy
features = ['alpha','delta','u','g','r','i','z','redshift']

for col in features:
    mean = data[col].mean()
    std = data[col].std()
    data = data[(data[col] >= mean - 3*std) & (data[col] <= mean + 3*std)]

le = LabelEncoder()
data['class_encoded'] = le.fit_transform(data['class'])  # GALAXY=0, STAR=1, QSO=2

min_count = data['class_encoded'].value_counts().min()
balanced_data = pd.concat([
    df.sample(min_count, random_state=42)
    for _, df in data.groupby('class_encoded')
])

# Funkcja split_data działa na macierzy numpy, więc tworzymy macierz danych
X = balanced_data[features].values
y_encoded = balanced_data['class_encoded'].values
y = np.eye(3)[y_encoded]  # one-hot

# Łączymy X i y, żeby podział był zsynchronizowany
combined = np.concatenate((X, y), axis=1)

# def split_data(data, train_ratio=0.6, validation_ratio=0.2):
#     np.random.shuffle(data)
#     train_size = int(len(data) * train_ratio)
#     validation_size = int(len(data) * validation_ratio)
#     train_data = data[:train_size]
#     validation_data = data[train_size:train_size + validation_size]
#     test_data = data[train_size + validation_size:]
#     return train_data, validation_data, test_data

# Podział danych na zbiory treningowe, generalizacyjne i walidacyjne
def split_data(data, train_ratio=0.6, validation_ratio=0.2):
    np.random.shuffle(data) # tasowanie danych
    
    train_size = int(len(data) * train_ratio) 
    validation_size = int(len(data) * validation_ratio)

    train_data = data[:train_size] # wybiera obserwacje do liczby "train_size"
    validation_data = data[train_size:train_size + validation_size] # wybiera obserwacje od "train_size" do sumy "train_size" i "validation_size"
    test_data = data[train_size + validation_size:] # wybiera obserwacje od powyzszej sumy do końca

    return train_data, validation_data, test_data

train_data, validation_data, test_data = split_data(combined)

# Oddzielamy cechy i klasy
X_train = train_data[:, :len(features)]
y_train = train_data[:, len(features):]
X_validation = validation_data[:, :len(features)]
y_validation = validation_data[:, len(features):]
X_test = test_data[:, :len(features)]
y_test = test_data[:, len(features):]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

# --- PODSUMOWANIE ---
print("Rozmiary zbiorów:")
print(f"Train: {X_train.shape}, Validation: {X_validation.shape}, Test: {X_test.shape}")

# Liczebność klas w train
train_classes = np.argmax(y_train, axis=1)
unique, counts = np.unique(train_classes, return_counts=True)
print("\nLiczebność klas w zbiorze treningowym:")
print(dict(zip(unique, counts)))


Rozmiary zbiorów:
Train: (30789, 8), Validation: (10263, 8), Test: (10263, 8)

Liczebność klas w zbiorze treningowym:
{0: 10330, 1: 10253, 2: 10206}


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Funkcja aktywacji: Sigmoid
def sigmoid(x, derivative=False):
    if derivative:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

def relu(x, derivative=False):
    if derivative:
        return (x > 0).astype(float)
    return np.maximum(0, x)

# Inicjalizacja wag sieci dla wielu warstw ukrytych
# def initialize_weights(input_size, hidden_layers_sizes, output_size):
#     weights = []
#     layer_sizes = [input_size] + hidden_layers_sizes + [output_size]
#     for i in range(len(layer_sizes) - 1):
#         weights.append(2 * np.random.random((layer_sizes[i], layer_sizes[i+1])) - 1)
#     return weights

def initialize_weights(input_size, hidden_layers_sizes, output_size):
    weights = []
    layer_sizes = [input_size] + hidden_layers_sizes + [output_size]
    for i in range(len(layer_sizes) - 1):
        weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2 / layer_sizes[i]))
    return weights

# Podział danych na zbiory treningowe, generalizacyjne i walidacyjne
def split_data(data, train_ratio=0.6, validation_ratio=0.2):
    np.random.shuffle(data) # tasowanie danych
    
    train_size = int(len(data) * train_ratio) 
    validation_size = int(len(data) * validation_ratio)

    train_data = data[:train_size] # wybiera obserwacje do liczby "train_size"
    validation_data = data[train_size:train_size + validation_size] # wybiera obserwacje od "train_size" do sumy "train_size" i "validation_size"
    test_data = data[train_size + validation_size:] # wybiera obserwacje od powyzszej sumy do końca

    return train_data, validation_data, test_data

# Funkcja dostosowująca tempa nauki
def adjust_learning_rate(learning_rate, mse, previous_mse, learning_rate_adjust, threshold=0.001):
    if mse < previous_mse:
        learning_rate *= 1.1  # jeśli błąd maleje, to zwiększa współczynnik uczenia o 10%
    else:
        learning_rate *= 0.5  # jeśli błąd nie maleje, to zmniejszamy współczynnik uczenia o 50%

    if np.abs(mse - previous_mse) < threshold: #jeśli różnica między błędami jest mniejsza niż dany próg to zmniejszamy learning rate, bo zbliżamy się do optymalnej konfiguracji
        learning_rate *= learning_rate_adjust

    return learning_rate

# Trening sieci z wieloma warstwami ukrytymi
def train(X, y, learning_rate, learning_rate_adjust, epochs, hidden_layers_sizes):
    input_size = X.shape[1]
    output_size = y.shape[1]
    weights = initialize_weights(input_size, hidden_layers_sizes, output_size)

    for epoch in range(epochs):
        # Forward pass
        activations = [X]
        zs = []
        for i, w in enumerate(weights):
            z = np.dot(activations[-1], w)
            if i == len(weights) - 1:
                activations.append(softmax(z))
            else:
                activations.append(relu(z))
        predicted_output = activations[-1]

        # Backpropagation
        error = predicted_output - y  # cross-entropy gradient
        deltas = [error] 
        for i in range(len(weights) - 1, 0, -1):
            delta = deltas[-1].dot(weights[i].T) * relu(activations[i], derivative=True)
            deltas.append(delta)

        deltas.reverse() 

        # Aktualizacja wag
        # for i in range(len(weights)):
        #     weights[i] += activations[i].T.dot(deltas[i]) * learning_rate

        # # Dostosowanie learning rate
        # learning_rate = adjust_learning_rate(
        #     learning_rate,
        #     np.mean(error ** 2),
        #     np.mean((y - predicted_output) ** 2),
        #     learning_rate_adjust
        # )

        for i in range(len(weights)):
            weights[i] -= learning_rate * activations[i].T.dot(deltas[i]) / X.shape[0]

    return weights


def predict(X, weights):
    output = X
    for i, w in enumerate(weights):
        output = np.dot(output, w)
        if i < len(weights) - 1:
            output = relu(output)
        else:
            output = softmax(output)
    return output

# Funkcja obliczająca błąd predykcji
def calculate_error(predictions, labels):
    return np.mean(np.abs(predictions - labels))

def correct(y, predictions):
    # y i predictions są one-hot encoded
    y_class = np.argmax(y, axis=1)
    pred_class = np.argmax(predictions, axis=1)
    correct_predictions = np.sum(y_class == pred_class)
    return correct_predictions / len(y)

# Parametry sieci
learning_rates = [0.01]
learning_rate_adjusts = [0.0005]
epochses = [1000]
repeat = 3

# Warstwy
hidden_layers_sizes_list = [
    [10],         
    [10, 10]       
]

# Funkcja obliczająca wyniki dla accuracy, precision, recall, F1
def calculate_metrics(y_test_class, y_pred_class):
    accuracy = accuracy_score(y_test_class, y_pred_class)
    precision = precision_score(y_test_class, y_pred_class, average='macro')
    recall = recall_score(y_test_class, y_pred_class, average='macro')
    f1 = f1_score(y_test_class, y_pred_class, average='macro')
    
    return accuracy, precision, recall, f1

# Przechowywanie wyników dla różnych konfiguracji warstw
results = []

# Testowanie
for hidden_layers_sizes in hidden_layers_sizes_list:
    # Zbieramy metryki dla tej konkretnej konfiguracji warstw
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    
    best_accuracy = -np.inf
    best_precision = -np.inf
    best_recall = -np.inf
    best_f1 = -np.inf

  
    for r in range(1, repeat + 1):
        for lr in learning_rates:
            for lr_adj in learning_rate_adjusts:
                for epochs in epochses:
                        trained_weights = train(
                            X_train, y_train, lr, lr_adj, epochs, hidden_layers_sizes
                        )
                        predictions_train = predict(X_train, trained_weights)
                        predictions_validation = predict(X_validation, trained_weights)
                        predictions_test = predict(X_test, trained_weights)

                        # Zaokrąglamy wyniki do wartości 0 lub 1 dla porównań
                        # y_pred = (predictions_test > 0.5).astype(int)
                        y_pred_class = np.argmax(predictions_test, axis=1)
                        y_test_class = np.argmax(y_test, axis=1)

                        # Obliczamy metryki
                        accuracy, precision, recall, f1 = calculate_metrics(y_test_class, y_pred_class)

                        # Dodajemy metryki do list
                        accuracies.append(accuracy)
                        precisions.append(precision)
                        recalls.append(recall)
                        f1_scores.append(f1)

                        # Zbieramy najlepsze wyniki
                        best_accuracy = max(best_accuracy, accuracy)
                        best_precision = max(best_precision, precision)
                        best_recall = max(best_recall, recall)
                        best_f1 = max(best_f1, f1)

    # Obliczanie średnich wartości dla tej konfiguracji
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    # Dodanie wyników do tabeli
    results.append({
        'hidden_layers': hidden_layers_sizes,
        'avg_accuracy': avg_accuracy,
        'avg_precision': avg_precision,
        'avg_recall': avg_recall,
        'avg_f1': avg_f1,
        'best_accuracy': best_accuracy,
        'best_precision': best_precision,
        'best_recall': best_recall,
        'best_f1': best_f1
    })

# Tworzenie DataFrame z wynikami
results_df = pd.DataFrame(results)

# Wyświetlanie wyników
print(results_df)

  hidden_layers  avg_accuracy  avg_precision  avg_recall    avg_f1  \
0          [10]      0.796876       0.805232    0.794130  0.790145   
1      [10, 10]      0.807821       0.817255    0.806535  0.805625   

   best_accuracy  best_precision  best_recall   best_f1  
0       0.814674        0.826860     0.812373  0.811486  
1       0.823931        0.832897     0.822727  0.823841  


In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

# Funkcja do obliczania metryk globalnych i per klasę
def calculate_classwise_metrics(y_test_class, y_pred_class):
    accuracy = accuracy_score(y_test_class, y_pred_class)
    precision_macro = precision_score(y_test_class, y_pred_class, average='macro')
    recall_macro = recall_score(y_test_class, y_pred_class, average='macro')
    f1_macro = f1_score(y_test_class, y_pred_class, average='macro')

    precision_per_class = precision_score(y_test_class, y_pred_class, average=None)
    recall_per_class = recall_score(y_test_class, y_pred_class, average=None)
    f1_per_class = f1_score(y_test_class, y_pred_class, average=None)

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class
    }

# Zakładamy, że masz N klas
n_classes = y_train.shape[1]  # liczba klas w danych

results = []

for hidden_layers_sizes in hidden_layers_sizes_list:
    for r in range(1, repeat + 1):
        for lr in learning_rates:
            for lr_adj in learning_rate_adjusts:
                for epochs in epochses:
                    trained_weights = train(X_train, y_train, lr, lr_adj, epochs, hidden_layers_sizes)
                    predictions_test = predict(X_test, trained_weights)
                    y_pred_class = np.argmax(predictions_test, axis=1)
                    y_test_class = np.argmax(y_test, axis=1)

                    metrics = calculate_classwise_metrics(y_test_class, y_pred_class)

                    # Tworzymy słownik do DataFrame
                    row = {
                        'hidden_layers': str(hidden_layers_sizes),
                        'accuracy': metrics['accuracy'],
                        'precision_macro': metrics['precision_macro'],
                        'recall_macro': metrics['recall_macro'],
                        'f1_macro': metrics['f1_macro'],
                    }

                    # Dodajemy metryki dla każdej klasy
                    for i in range(n_classes):
                        row[f'precision_class_{i}'] = metrics['precision_per_class'][i]
                        row[f'recall_class_{i}'] = metrics['recall_per_class'][i]
                        row[f'f1_class_{i}'] = metrics['f1_per_class'][i]

                    results.append(row)


results_df = pd.DataFrame(results)

# Średnie metryki dla każdej konfiguracji warstw
summary_df = results_df.groupby('hidden_layers').mean().reset_index()

pd.set_option('display.float_format', '{:.4f}'.format)
print(summary_df)


  hidden_layers  accuracy  precision_macro  recall_macro  f1_macro  \
0      [10, 10]    0.8268           0.8305        0.8257    0.8270   
1          [10]    0.7993           0.8033        0.7977    0.7983   

   precision_class_0  recall_class_0  f1_class_0  precision_class_1  \
0             0.7589          0.7710      0.7644             0.9616   
1             0.7381          0.6928      0.7144             0.9405   

   recall_class_1  f1_class_1  precision_class_2  recall_class_2  f1_class_2  
0          0.8884      0.9235             0.7709          0.8178      0.7932  
1          0.8642      0.9007             0.7314          0.8362      0.7799  


Sieć z biblioteki

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# =======================
# Parametry symulacji
# =======================
learning_rates = [0.01]
learning_rate_adjusts = [0.5]
epochses = [1000]
repeat = 3
hidden_layers_sizes_list = [[10], [10, 10]]

# =======================
# Wczytanie danych
# =======================
data = pd.read_csv('star_classification.csv', delimiter=",")

features = ['alpha','delta','u','g','r','i','z','redshift']

# Usuwanie wartości odstających
for col in features:
    mean = data[col].mean()
    std = data[col].std()
    data = data[(data[col] >= mean - 3*std) & (data[col] <= mean + 3*std)]

# Kodowanie klas
le = LabelEncoder()
data['class_encoded'] = le.fit_transform(data['class'])  # GALAXY=0, STAR=1, QSO=2

# Balansowanie klas
min_count = data['class_encoded'].value_counts().min()
balanced_data = pd.concat([
    df.sample(min_count, random_state=42)
    for _, df in data.groupby('class_encoded')
])

# Dane i etykiety
X = balanced_data[features].values
y = balanced_data['class_encoded'].values  # WEKTOR etykiet, nie one-hot

# Łączymy X i y, żeby podział był zsynchronizowany
combined = np.concatenate((X, y.reshape(-1,1)), axis=1)

# =======================
# Podział na zbiory
# =======================
def split_data(data, train_ratio=0.6, validation_ratio=0.2):
    np.random.shuffle(data)
    train_size = int(len(data) * train_ratio)
    validation_size = int(len(data) * validation_ratio)
    train_data = data[:train_size]
    validation_data = data[train_size:train_size + validation_size]
    test_data = data[train_size + validation_size:]
    return train_data, validation_data, test_data

train_data, validation_data, test_data = split_data(combined)

X_train = train_data[:, :-1]
y_train = train_data[:, -1].astype(int)
X_validation = validation_data[:, :-1]
y_validation = validation_data[:, -1].astype(int)
X_test = test_data[:, :-1]
y_test = test_data[:, -1].astype(int)

# Skalowanie
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

results = []

for lr in learning_rates:
    for lr_adj in learning_rate_adjusts:
        adjusted_lr = lr * lr_adj
        for epochs in epochses:
            for hidden_layers in hidden_layers_sizes_list:
                for r in range(repeat):
                    mlp = MLPClassifier(
                        hidden_layer_sizes=hidden_layers,
                        activation='logistic',
                        max_iter=epochs,
                        learning_rate_init=adjusted_lr,
                        random_state=r
                    )
                    mlp.fit(X_train, y_train)
                    
                    y_pred = mlp.predict(X_test)
                    
                    accuracy = accuracy_score(y_test, y_pred)
                    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
                    recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
                    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
                    
                    results.append({
                        "learning_rate": lr,
                        "lr_adjust": lr_adj,
                        "adjusted_lr": adjusted_lr,
                        "epochs": epochs,
                        "hidden_layers": hidden_layers,
                        "repeat": r+1,
                        "accuracy": accuracy,
                        "precision": precision,
                        "recall": recall,
                        "f1_score": f1
                    })

df_results = pd.DataFrame(results)
print(df_results)

# Opcjonalnie zapis
# df_results.to_csv("mlp_results.csv", index=False)


   learning_rate  lr_adjust  adjusted_lr  epochs hidden_layers  repeat  \
0           0.01        0.5        0.005    1000          [10]       1   
1           0.01        0.5        0.005    1000          [10]       2   
2           0.01        0.5        0.005    1000          [10]       3   
3           0.01        0.5        0.005    1000      [10, 10]       1   
4           0.01        0.5        0.005    1000      [10, 10]       2   
5           0.01        0.5        0.005    1000      [10, 10]       3   

   accuracy  precision    recall  f1_score  
0  0.966189   0.965981  0.966015  0.965935  
1  0.966774   0.966539  0.966592  0.966521  
2  0.964533   0.964295  0.964335  0.964249  
3  0.965994   0.965881  0.965843  0.965746  
4  0.962487   0.962507  0.962331  0.962212  
5  0.966384   0.966143  0.966192  0.966116  


In [ ]:
# =======================
# Symulacje różnych parametrów z wynikami po klasach
# =======================
results = []

for lr in learning_rates:
    for lr_adj in learning_rate_adjusts:
        adjusted_lr = lr * lr_adj
        for epochs in epochses:
            for hidden_layers in hidden_layers_sizes_list:
                for r in range(repeat):
                    mlp = MLPClassifier(
                        hidden_layer_sizes=hidden_layers,
                        activation='logistic',
                        max_iter=epochs,
                        learning_rate_init=adjusted_lr,
                        random_state=r
                    )
                    mlp.fit(X_train, y_train)
                    
                    y_pred = mlp.predict(X_test)
                    
                    # Ogólne miary
                    accuracy = accuracy_score(y_test, y_pred)
                    
                    # Miary dla poszczególnych klas
                    precision_classes = precision_score(y_test, y_pred, average=None, zero_division=0)
                    recall_classes = recall_score(y_test, y_pred, average=None, zero_division=0)
                    f1_classes = f1_score(y_test, y_pred, average=None, zero_division=0)
                    
                    results.append({
                        "learning_rate": lr,
                        "lr_adjust": lr_adj,
                        "adjusted_lr": adjusted_lr,
                        "epochs": epochs,
                        "hidden_layers": hidden_layers,
                        "repeat": r+1,
                        "accuracy": accuracy,
                        "precision_GALAXY": precision_classes[0],
                        "precision_STAR": precision_classes[1],
                        "precision_QSO": precision_classes[2],
                        "recall_GALAXY": recall_classes[0],
                        "recall_STAR": recall_classes[1],
                        "recall_QSO": recall_classes[2],
                        "f1_GALAXY": f1_classes[0],
                        "f1_STAR": f1_classes[1],
                        "f1_QSO": f1_classes[2]
                    })

df_results = pd.DataFrame(results)
print(df_results)

# Opcjonalnie zapis
# df_results.to_csv("mlp_results_by_class.csv", index=False)


   learning_rate  lr_adjust  adjusted_lr  epochs hidden_layers  repeat  \
0           0.01        0.5        0.005    1000          [10]       1   
1           0.01        0.5        0.005    1000          [10]       2   
2           0.01        0.5        0.005    1000          [10]       3   
3           0.01        0.5        0.005    1000      [10, 10]       1   
4           0.01        0.5        0.005    1000      [10, 10]       2   
5           0.01        0.5        0.005    1000      [10, 10]       3   

   accuracy  precision_GALAXY  precision_STAR  precision_QSO  recall_GALAXY  \
0  0.966189          0.943890        0.966607       0.987447       0.953695   
1  0.966774          0.946623        0.965548       0.987447       0.952805   
2  0.964533          0.943346        0.966021       0.983518       0.948946   
3  0.965994          0.940233        0.970243       0.987165       0.957257   
4  0.962487          0.932985        0.973260       0.981277       0.954586   
5  0.96